# The Beermile

## Retrieve Data

In [ ]:
import sys
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [ ]:
url_base = 'https://www.beermile.com/display/'
max_event_id = 151381

In [ ]:
%%time

# Find which events are actually races
is_event = 0
not_event = 0


# Look for all events by numerical ID
for i in range(1, max_event_id):
    # Website uses sequential 
    event = f'event_{i}'
    url = f'{url_base}{event}'
    result = requests.get(url)
    page = urlopen(result.url).read()
    a = bs(page, 'html.parser')
    
    eventType = False
    if (a.title.string == 'Race Not Available'):
        not_event += 1
        with open('not_events.txt', 'a') as g:
            g.write(f'{i}\n')
    else:
        is_event += 1
        eventType = True
        with open('events.txt', 'a') as f:
            f.write(f'{i}\n')
            
    print(f'\r #{i:06d} Events: {is_event:<6} - Not Events: {not_event:<6}', end='') 
    sys.stdout.flush()


## Clean Data Into DataFrame

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [ ]:
def cleanSoupList(soup_list):
    clean_soup_list = [s.text.strip() for s in soup_list]
    return clean_soup_list

def getNewCols(headers, columns=[]):
    '''
    Use headers (from a table) Soup to find columns not already defined  
    '''
    new_columns = []
    headers = cleanSoupList(headers)
    for col in headers:
        if col not in columns:
            new_columns.append(col)
    return new_columns

def getDataRows(event_rows):
    data = []
    is_header = True
    for row in event_rows:
        # Skip over first row (since it's the header)
        if (is_header):
            is_header = False
        else:
            cols = row.find_all('td')
            # TODO: Determine how to "clean" string
            cols = [ele.text.strip() for ele in cols]
            row = [ele for ele in cols]
            data.append(row)
    return data

def addToDf(events_soup, df):
    '''
    Add data (and potentially new columns) from Soup to df
    '''
    # Add new columns to df
    columns = df.columns
    headers = events_soup.find_all('tr')[0].find_all('td')
    new_columns = getNewCols(headers, columns)
    for c in new_columns:
        df[c] = np.nan
    # Add data to df
    data_rows = events_soup.find_all('tr')
    new = getDataRows(data_rows)
    df =df.append(
        pd.DataFrame(new, columns=cleanSoupList(headers))
    )
    
    return df

### Iterate through events list to get data into dataframe

In [ ]:
count = 0
url_event_base = 'https://www.beermile.com/display/event_'
cols = []

df = pd.DataFrame()

for n_event in open('events.txt'):
    url = f'{url_event_base}{n_event}'
    page = urlopen(url).read()
    soup = bs(page, 'html.parser')
    
    df = addToDf(soup,df)
    # TEST: Only go through 5 results
    count += 1
    if count >5:
        break

# Display DataFrame created
display(df.head())

In [ ]:
df.describe()